In [13]:
import pandas as pd
import os
from scipy import stats
import numpy as np
%matplotlib inline
import seaborn as sb
import matplotlib.pyplot as plt
import dask.dataframe as dd
from datetime import datetime
import random
import sys

In [2]:
raw = '../data/raw/'

In [17]:
#train = pd.read_csv(raw + 'train.csv', skiprows = np.arange(1, 125497040 - 10000), header = 0)
train = dd.read_csv(raw + 'train_2016.csv')
items = pd.read_csv(raw + 'items.csv')
holidays = pd.read_csv(raw + 'holidays_events.csv')
oil = pd.read_csv(raw + 'oil.csv')
stores = pd.read_csv(raw + 'stores.csv')
transactions = pd.read_csv(raw + 'transactions.csv')

#train, items, stores, transactions, and oil can be merged straightforwardly, for holidays we need to worry about

In [18]:
#train.to_csv(raw + 'train_last_10k.csv', index = False)

In [21]:
print('Train: ', train.columns, '\n\n Items: ', items.columns,
     '\n\n Holidays: ', holidays.columns, '\n\n Oil:', oil.columns, 
        '\n\n Stores: ', stores.columns,
     '\n\n Transactions: ', transactions.columns)

Train:  Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'], dtype='object') 

 Items:  Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object') 

 Holidays:  Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object') 

 Oil: Index(['date', 'dcoilwtico'], dtype='object') 

 Stores:  Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object') 

 Transactions:  Index(['date', 'store_nbr', 'transactions'], dtype='object')


In [22]:
d = train.merge(items)
d = d.merge(oil)
d = d.merge(stores)
d = d.merge(transactions)

In [23]:
d.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,dcoilwtico,city,state,type,cluster,transactions
0,66458907,2015-12-31,54,2027252,4.0,False,GROCERY I,1034,0,37.13,El Carmen,Manabi,C,3,1572
1,66852843,2016-01-05,54,2027252,6.0,False,GROCERY I,1034,0,35.97,El Carmen,Manabi,C,3,866
2,66851649,2016-01-05,54,105857,1.0,False,GROCERY I,1092,0,35.97,El Carmen,Manabi,C,3,866
3,66851651,2016-01-05,54,108701,1.0,False,DELI,2644,1,35.97,El Carmen,Manabi,C,3,866
4,66851652,2016-01-05,54,108786,4.0,False,CLEANING,3044,0,35.97,El Carmen,Manabi,C,3,866


In [24]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [25]:
pd.crosstab(holidays['locale'], holidays['locale_name'])

locale_name,Ambato,Cayambe,Cotopaxi,Cuenca,Ecuador,El Carmen,Esmeraldas,Guaranda,Guayaquil,Ibarra,...,Machala,Manta,Puyo,Quevedo,Quito,Riobamba,Salinas,Santa Elena,Santo Domingo,Santo Domingo de los Tsachilas
locale,,,,,,,,,,,,,,,,,,,,,
Local,12,6,0,7,0,6,6,12,11,7,...,6,6,6,6,13,12,6,0,6,0
National,0,0,0,0,174,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Regional,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6,0,6


In [26]:
stores.city.unique()

array(['Quito', 'Santo Domingo', 'Cayambe', 'Latacunga', 'Riobamba',
       'Ibarra', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil', 'Salinas',
       'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad', 'Cuenca',
       'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'], dtype=object)

In [27]:
stores.state.unique()

array(['Pichincha', 'Santo Domingo de los Tsachilas', 'Cotopaxi',
       'Chimborazo', 'Imbabura', 'Bolivar', 'Pastaza', 'Tungurahua',
       'Guayas', 'Santa Elena', 'Los Rios', 'Azuay', 'Loja', 'El Oro',
       'Esmeraldas', 'Manabi'], dtype=object)

In [28]:
#okay, so local means city, regional means state, and national means 'everywhere'
d['holitype'] = 'No_Holiday'
nat_n = 0
reg_n = 0
loc_n = 0
for i in range(0, len(d['holitype']) -1):
    if d['date'][i] in holidays['date'].values:
        this_day = holidays[holidays.date == d['date'][i]]
        if 'National' in this_day['locale'].values:
            #d['holitype'][i] = this_day['type']
            #print(this_day[this_day.locale == 'National'])
            #print('National')
            nat_n += 1
        elif 'Regional' in this_day['locale'].values:
            #print('Regional')
            reg_n += 1
        elif 'Local' in this_day['locale'].values:
            #print('Local')
            loc_n +=1
print('National: ', nat_n)
print('Regional: ', reg_n)
print('Local: ', loc_n)
# hmm, for this data subset we only have local holidays

NotImplementedError: 

In [29]:
d['date'][0]
#guess that makes sense since we are starting in August, eek! Lets get all of a year at least!

NotImplementedError: 

In [3]:
#dates_only = pd.read_csv(raw + 'train.csv', usecols = ['date'])

In [4]:
#dates_2016 = dates_only[['2016' in x for x in dates_only.date]].index
#print('First: ', dates_2016[0])

First:  66458908


In [17]:
#skip = list(range(1, dates_2016[0]))

In [18]:
#skip += (list(range(dates_2016[-1] + 1, 125497041)))

In [19]:
#skip2 = [x for x in skip if type(x) == int ]

In [20]:
#train_2016 = pd.read_csv(raw + 'train.csv', skiprows = skip2)

In [24]:
#train_2016.to_csv(raw + 'train_2016.csv', index = False)

In [32]:
train.visualize()

ExecutableNotFound: failed to execute ['dot', '-Tpng'], make sure the Graphviz executables are on your systems' PATH